In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [6]:
train = pd.read_csv(r"C:\Users\user\Videos\Datasets\Flight Delay Prediction\Train (10).csv")
test = pd.read_csv(r"C:\Users\user\Videos\Datasets\Flight Delay Prediction\Test (7).csv")
sample_submission = pd.read_csv(r"C:\Users\user\Videos\Datasets\Flight Delay Prediction\SampleSubmission (5).csv")

In [7]:
train.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0


In [8]:
train.columns

Index(['ID', 'DATOP', 'FLTID', 'DEPSTN', 'ARRSTN', 'STD', 'STA', 'STATUS',
       'AC', 'target'],
      dtype='object')

In [9]:
test.columns

Index(['ID', 'DATOP', 'FLTID', 'DEPSTN', 'ARRSTN', 'STD', 'STA', 'STATUS',
       'AC'],
      dtype='object')

In [10]:
train.ID.duplicated().sum()

0

In [11]:
test.ID.duplicated().sum()

0

In [12]:
train.FLTID.duplicated().sum()

105972

In [13]:
len(train.FLTID.unique())

1861

In [14]:
train.shape, test.shape

((107833, 10), (9333, 9))

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      107833 non-null  object 
 1   DATOP   107833 non-null  object 
 2   FLTID   107833 non-null  object 
 3   DEPSTN  107833 non-null  object 
 4   ARRSTN  107833 non-null  object 
 5   STD     107833 non-null  object 
 6   STA     107833 non-null  object 
 7   STATUS  107833 non-null  object 
 8   AC      107833 non-null  object 
 9   target  107833 non-null  float64
dtypes: float64(1), object(9)
memory usage: 8.2+ MB


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9333 entries, 0 to 9332
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      9333 non-null   object
 1   DATOP   9333 non-null   object
 2   FLTID   9333 non-null   object
 3   DEPSTN  9333 non-null   object
 4   ARRSTN  9333 non-null   object
 5   STD     9333 non-null   object
 6   STA     9333 non-null   object
 7   STATUS  9333 non-null   object
 8   AC      9333 non-null   object
dtypes: object(9)
memory usage: 656.4+ KB


In [17]:
import datetime
train["STA"] = train["STA"].map(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H.%M.%S").strftime("%Y-%m-%d %H:%M:%S"))
test["STA"] = test["STA"].map(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H.%M.%S").strftime("%Y-%m-%d %H:%M:%S"))

In [18]:
train["STA"].head()

0    2016-01-03 12:55:00
1    2016-01-13 16:55:00
2    2016-01-16 06:45:00
3    2016-01-17 17:00:00
4    2016-01-17 15:50:00
Name: STA, dtype: object

In [19]:
for i in ['DATOP','STD','STA']:
    train[i] = pd.to_datetime(train[i])
    test[i] = pd.to_datetime(test[i])

In [20]:
# The markers for Parts ofthe day used are {1 - "Morning", 2- "Afternoon", 3- "Evening", 4- "Night"}
# The markers for the seasons of the year are {1 - "Winter", 2 - "Spring", 3 - "Summer", 4 - "Fall"}

for i in ["STD","STA","DATOP"]:
    train[i+"_month"] = pd.DatetimeIndex(train[i]).month
    train[i+"_year"] = pd.DatetimeIndex(train[i]).year
    train[i+"_day_of_week"] = pd.DatetimeIndex(train[i]).day_of_week
    train[i+"_is_year_end"] = pd.DatetimeIndex(train[i]).is_year_end
    train[i+"_is_year_start"] = pd.DatetimeIndex(train[i]).is_year_start
    train[i+"_Seasons"] = train[i+"_month"] % 12 // 3 + 1


    def get_part_of_day(hour):
        return (
            1 if 5 <= hour <= 11
            else
            2 if 12 <= hour <= 17
            else
            3 if 18 <= hour <= 22
            else
            4)

    train[i+"_Hour"] = pd.DatetimeIndex(train[i]).hour
    train[i+'part_of_day'] = train[i+"_Hour"].map(get_part_of_day)

In [21]:
train["departure_time"] = (train["STD"] - train["DATOP"]) / np.timedelta64(1, "m")
train["diff_in dept&arr_time"] = (train["STA"] - train["STD"]) / np.timedelta64(1, "m")
train["arrival_time"] = (train["STA"] - train["DATOP"]) / np.timedelta64(1,"m")

In [22]:
for i in ["STD","STA","DATOP"]:
    test[i+"_month"] = pd.DatetimeIndex(test[i]).month
    test[i+"_year"] = pd.DatetimeIndex(test[i]).year
    test[i+"_day_of_week"] = pd.DatetimeIndex(test[i]).day_of_week
    test[i+"_is_year_end"] = pd.DatetimeIndex(test[i]).is_year_end
    test[i+"_is_year_start"] = pd.DatetimeIndex(test[i]).is_year_start
    test[i+"_Seasons"] = test[i+"_month"] % 12 // 3 + 1


    def get_part_of_day(hour):
        return (
            1 if 5 <= hour <= 11
            else
            2 if 12 <= hour <= 17
            else
            3 if 18 <= hour <= 22
            else
            4)

    test[i+"_Hour"] = pd.DatetimeIndex(test[i]).hour
    test[i+'part_of_day'] = test[i+"_Hour"].map(get_part_of_day)

In [23]:
test["departure_time"] = (test["STD"] - test["DATOP"]) / np.timedelta64(1, "m")
test["diff_in dept&arr_time"] = (test["STA"] - test["STD"]) / np.timedelta64(1, "m")
test["arrival_time"] = (test["STA"] - test["DATOP"]) / np.timedelta64(1,"m")

In [24]:
train.STATUS.unique()

array(['ATA', 'DEP', 'RTR', 'SCH', 'DEL'], dtype=object)

In [25]:
len(train.DEPSTN.unique())

132

In [26]:
len(train.ARRSTN.unique())

128

In [27]:
len(train.AC.unique())

68

In [28]:
from sklearn.preprocessing import LabelEncoder

col = ["STATUS", "DEPSTN","ARRSTN","AC","FLTID"] 

for i in col:
    Encoder = LabelEncoder()
    train[i] = Encoder.fit_transform(train[i])

In [29]:
for i in col:
    Encoder = LabelEncoder()
    test[i] = Encoder.fit_transform(test[i])

In [30]:
train.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,...,DATOP_year,DATOP_day_of_week,DATOP_is_year_end,DATOP_is_year_start,DATOP_Seasons,DATOP_Hour,DATOPpart_of_day,departure_time,diff_in dept&arr_time,arrival_time
0,train_id_0,2016-01-03,230,31,119,2016-01-03 10:30:00,2016-01-03 12:55:00,0,46,260.0,...,2016,6,False,False,1,0,4,630.0,145.0,775.0
1,train_id_1,2016-01-13,257,86,119,2016-01-13 15:05:00,2016-01-13 16:55:00,0,28,20.0,...,2016,2,False,False,1,0,4,905.0,110.0,1015.0
2,train_id_2,2016-01-16,86,123,58,2016-01-16 04:10:00,2016-01-16 06:45:00,0,46,0.0,...,2016,5,False,False,1,0,4,250.0,155.0,405.0
3,train_id_3,2016-01-17,164,37,91,2016-01-17 14:10:00,2016-01-17 17:00:00,0,50,0.0,...,2016,6,False,False,1,0,4,850.0,170.0,1020.0
4,train_id_4,2016-01-17,136,123,4,2016-01-17 14:30:00,2016-01-17 15:50:00,0,34,22.0,...,2016,6,False,False,1,0,4,870.0,80.0,950.0


In [31]:
test.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,STD_month,...,DATOP_year,DATOP_day_of_week,DATOP_is_year_end,DATOP_is_year_start,DATOP_Seasons,DATOP_Hour,DATOPpart_of_day,departure_time,diff_in dept&arr_time,arrival_time
0,test_id_0,2016-05-04,124,23,76,2016-05-04 06:40:00,2016-05-04 07:30:00,0,22,5,...,2016,2,False,False,2,0,4,400.0,50.0,450.0
1,test_id_1,2016-05-05,53,74,9,2016-05-05 15:20:00,2016-05-05 20:05:00,0,19,5,...,2016,3,False,False,2,0,4,920.0,285.0,1205.0
2,test_id_2,2016-05-06,151,29,76,2016-05-06 10:00:00,2016-05-06 12:25:00,0,20,5,...,2016,4,False,False,2,0,4,600.0,145.0,745.0
3,test_id_3,2016-05-11,172,7,76,2016-05-11 09:40:00,2016-05-11 13:10:00,0,12,5,...,2016,2,False,False,2,0,4,580.0,210.0,790.0
4,test_id_4,2016-05-11,109,62,50,2016-05-11 09:50:00,2016-05-11 12:35:00,0,37,5,...,2016,2,False,False,2,0,4,590.0,165.0,755.0


In [32]:
train_data = train.drop(labels = ["ID","DATOP","STD","STA"], axis = 1)

In [33]:
test_data = test.drop(labels = ["ID","DATOP","STD","STA"], axis = 1)

In [34]:
train_data.head()

,FLTID,DEPSTN,ARRSTN,STATUS,AC,target,STD_month,STD_year,STD_day_of_week,STD_is_year_end,...,DATOP_year,DATOP_day_of_week,DATOP_is_year_end,DATOP_is_year_start,DATOP_Seasons,DATOP_Hour,DATOPpart_of_day,departure_time,diff_in dept&arr_time,arrival_time
0,230,31,119,0,46,260.0,1,2016,6,False,...,2016,6,False,False,1,0,4,630.0,145.0,775.0
1,257,86,119,0,28,20.0,1,2016,2,False,...,2016,2,False,False,1,0,4,905.0,110.0,1015.0
2,86,123,58,0,46,0.0,1,2016,5,False,...,2016,5,False,False,1,0,4,250.0,155.0,405.0
3,164,37,91,0,50,0.0,1,2016,6,False,...,2016,6,False,False,1,0,4,850.0,170.0,1020.0
4,136,123,4,0,34,22.0,1,2016,6,False,...,2016,6,False,False,1,0,4,870.0,80.0,950.0


In [35]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   FLTID                  107833 non-null  int32  
 1   DEPSTN                 107833 non-null  int32  
 2   ARRSTN                 107833 non-null  int32  
 3   STATUS                 107833 non-null  int32  
 4   AC                     107833 non-null  int32  
 5   target                 107833 non-null  float64
 6   STD_month              107833 non-null  int64  
 7   STD_year               107833 non-null  int64  
 8   STD_day_of_week        107833 non-null  int64  
 9   STD_is_year_end        107833 non-null  bool   
 10  STD_is_year_start      107833 non-null  bool   
 11  STD_Seasons            107833 non-null  int64  
 12  STD_Hour               107833 non-null  int64  
 13  STDpart_of_day         107833 non-null  int64  
 14  STA_month              107833 non-nu

In [36]:
for i in train_data.columns:
    if train_data[i].dtype == "bool":
        train_data[i] = Encoder.fit_transform(train[i])

In [37]:
for i in test_data.columns:
    if test_data[i].dtype == "bool":
        test_data[i] = Encoder.fit_transform(test[i])

In [38]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107833 entries, 0 to 107832
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   FLTID                  107833 non-null  int32  
 1   DEPSTN                 107833 non-null  int32  
 2   ARRSTN                 107833 non-null  int32  
 3   STATUS                 107833 non-null  int32  
 4   AC                     107833 non-null  int32  
 5   target                 107833 non-null  float64
 6   STD_month              107833 non-null  int64  
 7   STD_year               107833 non-null  int64  
 8   STD_day_of_week        107833 non-null  int64  
 9   STD_is_year_end        107833 non-null  int64  
 10  STD_is_year_start      107833 non-null  int64  
 11  STD_Seasons            107833 non-null  int64  
 12  STD_Hour               107833 non-null  int64  
 13  STDpart_of_day         107833 non-null  int64  
 14  STA_month              107833 non-nu

In [39]:
X = train_data[[i for i in train_data.columns if i != "target"]]
y = np.log1p(train_data["target"])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
training = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [40]:
y.isnull().sum()

0

In [41]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error

In [42]:
from catboost import *

In [43]:
from sklearn.model_selection import KFold, StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1235)
oofs = np.zeros((len(X)))
preds = np.zeros((len(test_data)))

from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score, mean_squared_error, mean_absolute_error

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, pd.qcut(y, 10, labels=False, duplicates='drop'))):
    
    X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    

    clf = CatBoostRegressor(learning_rate=0.03, depth=5, use_best_model=True, l2_leaf_reg=0.2, colsample_bylevel=0.7)
    clf.fit(X_trn, y_trn, eval_set=[(X_trn, y_trn),(X_val, y_val)], early_stopping_rounds=100,verbose=500)
    
    vp = clf.predict(X_val)
    val_score = mean_absolute_error(np.expm1(vp), np.expm1(y_val))
    
    print(f'\nFold {fold_+1} Val score: {val_score}\n')
    
    tp = clf.predict(test_data)
    
    oofs[val_idx] = vp
    preds += tp/10

    
oof_score = mean_absolute_error(np.expm1(y), np.expm1(oofs))
print(f'\nOOF MAE is : {oof_score}')

0:	learn: 1.9340367	test: 1.9340367	test1: 1.9415015	best: 1.9415015 (0)	total: 185ms	remaining: 3m 5s
500:	learn: 1.5142497	test: 1.5142497	test1: 1.5156159	best: 1.5156159 (500)	total: 28.9s	remaining: 28.8s
999:	learn: 1.4794635	test: 1.4794635	test1: 1.4876956	best: 1.4876956 (999)	total: 50.4s	remaining: 0us

bestTest = 1.487695555
bestIteration = 999


Fold 1 Val score: 40.759164571389285

0:	learn: 1.9353481	test: 1.9353481	test1: 1.9310658	best: 1.9310658 (0)	total: 31.5ms	remaining: 31.5s
500:	learn: 1.5135890	test: 1.5135890	test1: 1.5261208	best: 1.5261208 (500)	total: 19.8s	remaining: 19.7s
999:	learn: 1.4788852	test: 1.4788852	test1: 1.4963889	best: 1.4963889 (999)	total: 38.3s	remaining: 0us

bestTest = 1.496388854
bestIteration = 999


Fold 2 Val score: 40.49885061041092

0:	learn: 1.9349865	test: 1.9349865	test1: 1.9337159	best: 1.9337159 (0)	total: 29.8ms	remaining: 29.8s
500:	learn: 1.5161020	test: 1.5161020	test1: 1.5087822	best: 1.5087822 (500)	total: 19.5s	remainin

In [44]:
rmse = np.sqrt(mean_squared_error(np.expm1(y), np.expm1(oofs)))
rmse

117.27259902509094

In [45]:
import xgboost as xgb
xgb_acc_list = []
kf = KFold(n_splits=5, shuffle=True, random_state=1)
  
for train_index, test_index in kf.split(X,y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    xgboost = xgb.XGBRegressor(n_estimators = 350, max_depth = 5, learning_rate = 0.1, n_jobs = -1, random_state = 0)
    
    xgboost.fit(X_train, y_train)
    xgboost_pred = xgboost.predict(X_test)

    xgb_acc_list.append(np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(xgboost_pred))))

In [46]:
np.mean(xgb_acc_list)

115.75480213958903

In [47]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [53]:
params = {"n_estimators": range(100,1000,100),
          "max_depth": range(3,15,2),
          "learning_rate": np.arange(0,1,0.1)}
# kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
grid_xgb = RandomizedSearchCV(xgb.XGBRegressor(random_state = 0), param_distributions = params, cv = 5 , n_jobs = -1)
grid_xgb.fit(X_train, y_train)
grid_xgb.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=400, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [49]:
import lightgbm as lgbm

In [50]:
lgb_acc_list = []
kf = KFold(n_splits=5, shuffle=True, random_state=1)
  
for train_index, test_index in kf.split(X,y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    lgb = lgbm.LGBMRegressor(learning_rate=0.2, max_depth=10, n_estimators=500, num_leaves=8,
                             n_jobs = -1, random_state = 0)
    
    lgb.fit(X_train, y_train)
    lgb_pred = lgb.predict(X_test)

    lgb_acc_list.append(np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(lgb_pred))))

In [51]:
np.mean(lgb_acc_list)

115.98544017484906

In [52]:
params = {"n_estimators": range(100,1000,100),
          "max_depth": range(3,15,2),
          "learning_rate": np.arange(0,1,0.05),
          "num_leaves": range(2,10)}
# kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
grid_lgbm = RandomizedSearchCV(lgbm.LGBMRegressor(random_state = 0), param_distributions = params, cv = 5 , n_jobs = -1)
grid_lgbm.fit(X_train, y_train)
grid_lgbm.best_estimator_

LGBMRegressor(learning_rate=0.15000000000000002, max_depth=3, n_estimators=800,
              num_leaves=8, random_state=0)

In [ ]:
params = {"n_estimators": range(450,600,50),
          "max_depth": range(10,15),
          "learning_rate": np.arange(0.2,0.4,0.05),
          "num_leaves": range(5,10)}
# kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
grid_lgbm = GridSearchCV(lgbm.LGBMRegressor(random_state = 0), param_grid = params, cv = 5 , 
                         n_jobs = -1, scoring = "neg_mean_squared_error")
grid_lgbm.fit(X_train, y_train)
grid_lgbm.best_estimator_

In [57]:
sample_submission.head()

,ID,target
0,test_id_0,2470
1,test_id_1,2944
2,test_id_2,2585
3,test_id_3,3264
4,test_id_4,1369


In [60]:
sample_submission["target"] = np.expm1(xgboost.predict(test_data))

In [61]:
sample_submission["target"].head()

0    64.110542
1     1.421101
2    33.753216
3     6.093468
4     4.573025
Name: target, dtype: float32

In [63]:
sample_submission.to_csv("submission", index = False)

In [64]:
import pickle
filename = 'flight_pred.pkl'
pickle.dump(xgboost, open(filename, 'wb'))